# Rf 2 0 7_Comptools
Addition and convolution: tools and utilities for manipulation of composite objects



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:44 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooChebychev.h"
#include "RooExponential.h"
#include "RooAddPdf.h"
#include "RooPlot.h"
#include "RooCustomizer.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "TH1.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Setup composite  pdf, dataset
 --------------------------------------------------------

Declare observable x

In [3]:
RooRealVar x("x", "x", 0, 10);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Create two gaussian pdfs g1(x,mean1,sigma) anf g2(x,mean2,sigma) and their parameters

In [4]:
RooRealVar mean("mean", "mean of gaussians", 5);
RooRealVar sigma("sigma", "width of gaussians", 0.5);
RooGaussian sig("sig", "Signal component 1", x, mean, sigma);

Build chebychev polynomial p.d.f.

In [5]:
RooRealVar a0("a0", "a0", 0.5, 0., 1.);
RooRealVar a1("a1", "a1", 0.2, 0., 1.);
RooChebychev bkg1("bkg1", "Background 1", x, RooArgSet(a0, a1));

Build exponential pdf

In [6]:
RooRealVar alpha("alpha", "alpha", -1);
RooExponential bkg2("bkg2", "Background 2", x, alpha);

Sum the background components into a composite background p.d.f.

In [7]:
RooRealVar bkg1frac("bkg1frac", "fraction of component 1 in background", 0.2, 0., 1.);
RooAddPdf bkg("bkg", "Signal", RooArgList(bkg1, bkg2), bkg1frac);

Sum the composite signal and background

In [8]:
RooRealVar bkgfrac("bkgfrac", "fraction of background", 0.5, 0., 1.);
RooAddPdf model("model", "g1+g2+a", RooArgList(bkg, sig), bkgfrac);

Create dummy dataset that has more observables than the above pdf

In [9]:
RooRealVar y("y", "y", -10, 10);
RooDataSet data("data", "data", RooArgSet(x, y));

---------------------------------------------------
Basic information requests
 ===================================================

Get list of observables
 ---------------------------------------------

Get list of observables of pdf in context of a dataset

 Observables are define each context as the variables
 shared between a model and a dataset. In this case
 that is the variable 'x'

In [10]:
RooArgSet *model_obs = model.getObservables(data);
model_obs->Print("v");

  1) 0x7fc387aa4010 RooRealVar:: x = 5  L(0 - 10)  "x"


Get list of parameters
 -------------------------------------------

Get list of parameters, given list of observables

In [11]:
RooArgSet *model_params = model.getParameters(x);
model_params->Print("v");

  1) 0x7fc386800000 RooRealVar::       a0 = 0.5  L(0 - 1)  "a0"
  2) 0x7fc3868003a8 RooRealVar::       a1 = 0.2  L(0 - 1)  "a1"
  3) 0x7fc3867fd000 RooRealVar::    alpha = -1 C  L(-INF - +INF)  "alpha"
  4) 0x7fc3867fa000 RooRealVar:: bkg1frac = 0.2  L(0 - 1)  "fraction of component 1 in background"
  5) 0x7fc3867f7000 RooRealVar::  bkgfrac = 0.5  L(0 - 1)  "fraction of background"
  6) 0x7fc387aa43b8 RooRealVar::     mean = 5 C  L(-INF - +INF)  "mean of gaussians"
  7) 0x7fc387aa4760 RooRealVar::    sigma = 0.5 C  L(-INF - +INF)  "width of gaussians"


Get list of parameters, given a dataset
 (Gives identical results to operation above)

In [12]:
RooArgSet *model_params2 = model.getParameters(data);
model_params2->Print();

RooArgSet::parameters = (a0,a1,alpha,bkg1frac,bkgfrac,mean,sigma)


Get list of components
 -------------------------------------------

Get list of component objects, including top-level node

In [13]:
RooArgSet *model_comps = model.getComponents();
model_comps->Print("v");

  1) 0x7fc3867f73a8 RooAddPdf:: model[ bkgfrac * bkg + [%] * sig ] = 0.582695  "g1+g2+a"
  2) 0x7fc3867fa3a8 RooAddPdf::   bkg[ bkg1frac * bkg1 + [%] * bkg2 ] = 0.16539  "Signal"
  3) 0x7fc386800750 RooChebychev::  bkg1[ x=x coefficients=(a0,a1) ] = 0.8  "Background 1"
  4) 0x7fc3867fd3a8 RooExponential::  bkg2[ x=x c=alpha ] = 0.00673795  "Background 2"
  5) 0x7fc387aa4b08 RooGaussian::   sig[ x=x mean=mean sigma=sigma ] = 1  "Signal component 1"


-------------------------------------------------------------------------------
Modifications to structure of composites
 ===============================================================================

Create a second gaussian

In [14]:
RooRealVar sigma2("sigma2", "width of gaussians", 1);
RooGaussian sig2("sig2", "Signal component 1", x, mean, sigma2);

Create a sum of the original gaussian plus the new second gaussian

In [15]:
RooRealVar sig1frac("sig1frac", "fraction of component 1 in signal", 0.8, 0., 1.);
RooAddPdf sigsum("sigsum", "sig+sig2", RooArgList(sig, sig2), sig1frac);

Construct a customizer utility to customize model

In [16]:
RooCustomizer cust(model, "cust");

Instruct the customizer to replace node 'sig' with node 'sigsum'

In [17]:
cust.replaceArg(sig, sigsum);

Build a clone of the input pdf according to the above customization
 instructions. Each node that requires modified is clone so that the
 original pdf remained untouched. The name of each cloned node is that
 of the original node suffixed by the name of the customizer object

 The returned head node own all nodes that were cloned as part of
 the build process so when cust_clone is deleted so will all other
 nodes that were created in the process.

In [18]:
RooAbsPdf *cust_clone = (RooAbsPdf *)cust.build(kTRUE);

[#1] INFO:ObjectHandling -- RooCustomizer::build(model): tree node sig will be replaced by sigsum
[#1] INFO:ObjectHandling -- RooCustomizer::build(model) Branch node RooAddPdf::model cloned: depends on a replaced parameter
[#1] INFO:ObjectHandling -- RooCustomizer::build(model) Branch node sig is already replaced


Print structure of clone of model with sig->sigsum replacement.

In [19]:
cust_clone->Print("t");

delete cust_clone;

0x7fc36468ad30 RooAddPdf::model_cust = 0.582695 [Auto,Dirty] 
  0x7fc3867fa3a8/V- RooAddPdf::bkg = 0.16539 [Auto,Dirty] 
    0x7fc386800750/V- RooChebychev::bkg1 = 0.8 [Auto,Dirty] 
      0x7fc387aa4010/V- RooRealVar::x = 5
      0x7fc386800000/V- RooRealVar::a0 = 0.5
      0x7fc3868003a8/V- RooRealVar::a1 = 0.2
    0x7fc3867fa000/V- RooRealVar::bkg1frac = 0.2
    0x7fc3867fd3a8/V- RooExponential::bkg2 = 0.00673795 [Auto,Dirty] 
      0x7fc387aa4010/V- RooRealVar::x = 5
      0x7fc3867fd000/V- RooRealVar::alpha = -1
  0x7fc3867f7000/V- RooRealVar::bkgfrac = 0.5
  0x7fc37a3d93a8/V- RooAddPdf::sigsum = 1 [Auto,Dirty] 
    0x7fc387aa4b08/V- RooGaussian::sig = 1 [Auto,Dirty] 
      0x7fc387aa4010/V- RooRealVar::x = 5
      0x7fc387aa43b8/V- RooRealVar::mean = 5
      0x7fc387aa4760/V- RooRealVar::sigma = 0.5
    0x7fc37a3d9000/V- RooRealVar::sig1frac = 0.8
    0x7fc37a3dc3a8/V- RooGaussian::sig2 = 1 [Auto,Dirty] 
      0x7fc387aa4010/V- RooRealVar::x = 5
      0x7fc387aa43b8/V- RooRealVar: